In [1]:
#!pip install pandas-summary
#!pip install numpy
#!pip install scikit-learn
#!pip install keras
#!pip install tensorflow
#! pip install matplotlib
#!pip install isoweek
#!pip install pyarrow
#! pip install sklearn_pandas
#! pip install mlflow

# Datos estructurados - ITBA 2020

https://www.kaggle.com/c/datos-estructurados-itba-2020/overview

In [2]:
import numpy as np
import pandas as pd
#  pip install pandas_summary
from pandas_summary import DataFrameSummary

In [3]:
import os
PATH_PROCESSED = 'processed/'
# Check whether the specified path exists or not
isExist = os.path.exists(PATH_PROCESSED)
if not isExist:

    # Create a new directory because it does not exist
    os.makedirs(PATH_PROCESSED)
    print("The new directory is created!")

The new directory is created!


# Competencia de Kaggle

[ir a Kaggle](https://www.kaggle.com/c/datos-estructurados-itba-2020/overview)


# Métrica de la competencia

$$
\textrm{RMSPE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$


## Importamos dataset

Conclusiones de los archivos

Embeddings de 3 en occupation, verlo en grafica cuán cerca están
Genero: entrarlo como está
Zip: Embedding 2 nodos
Gender: 1 Embedding
Age: 4 embeddings
Release Date: 9 embeddings


Datos que determinaran el rating
* item: release date, [tipo pelicula]
* user: age, gender, occupation, zip code
* train/test: rating

Conversiones:
item.release date > mes-año
train.timestamp > mes-año ??


* u.data: user id | item id | rating | timestamp
* u.item: movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
* u.user: user id | age | gender | occupation | zip code
* u1..n: are 80%/20% splits of the u data
* ua/ub: split the u data exactly 10 ratings per user in the set.

In [4]:
PATH_PROCESSED='processed/'
PATH = 'dataset/'

user, occupation = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="|") for fname in ['user', 'occupation']]
info, train = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="\t") for fname in ['info', 'train']]
test = pd.read_csv(PATH + 'test.csv', low_memory=False, sep=",")
item = pd.read_csv(PATH + 'item.csv',  low_memory=False, encoding='latin-1', sep="|")

item.rename( columns={'movieId' :'movie_id', 'movieTitle' :'movie_title', 'release date' :'release_date', 'video release date' :'video_release_date', 'IMDb URL' :'imdb_url', 'unknown' :'unknown', 'Action' :'action', 'Adventure' :'adventure', 'Animation' :'animation', 'Children\'s' :'children', 'Comedy' :'comedy', 'Crime' :'crime', 'Documentary' :'documentary', 'Drama' :'drama', 'Fantasy' :'fantasy', 'Film-Noir' :'film-noir', 'Horror' :'horror', 'Musical' :'musical', 'Mystery' :'mystery', 'Romance' :'romance', 'Sci-Fi' :'sci-fi', 'Thriller' :'thriller', 'War' :'war', 'Western' :'western'}, inplace=True)

user.rename( columns={'user id' :'user_id', 'age' :'age', 'gender' :'gender', 'occupation' :'occupation', 'zip code' :'zip_code'}, inplace=True)

train.rename( columns={'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
test.rename( columns={'Id' :'id', 'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
#data = train + test
data = train

In [5]:
train

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [6]:
display(item.head())
display(DataFrameSummary(item).summary())

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
count,1682.0,NaN,NaN,0.0,NaN,1682.0,1682.0,1682.0,1682.0,1682.0,...,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0
mean,841.5,NaN,NaN,NaN,NaN,0.001189,0.149227,0.080262,0.02497,0.072533,...,0.01308,0.014269,0.054697,0.033294,0.036266,0.146849,0.060048,0.149227,0.042212,0.016052
std,485.695893,NaN,NaN,NaN,NaN,0.034473,0.356418,0.271779,0.156081,0.259445,...,0.11365,0.118632,0.227455,0.179456,0.187008,0.354061,0.237646,0.356418,0.201131,0.125714
min,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,421.25,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,841.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1261.75,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1682.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
counts,1682,1682,1681,0,1679,1682,1682,1682,1682,1682,...,1682,1682,1682,1682,1682,1682,1682,1682,1682,1682
uniques,1682,1664,240,0,1660,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [7]:
user.columns

Index(['user_id', 'age', 'gender', 'occupation', 'zip_code'], dtype='object')

In [8]:
display(user.head())
display(DataFrameSummary(user).summary())

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


,user_id,age,gender,occupation,zip_code
count,943.0,943.0,NaN,NaN,NaN
mean,472.0,34.051962,NaN,NaN,NaN
std,272.364951,12.19274,NaN,NaN,NaN
min,1.0,7.0,NaN,NaN,NaN
25%,236.5,25.0,NaN,NaN,NaN
50%,472.0,31.0,NaN,NaN,NaN
75%,707.5,43.0,NaN,NaN,NaN
max,943.0,73.0,NaN,NaN,NaN
counts,943,943,943,943,943
uniques,943,61,2,21,795


- Mirar counts que todos tienen la misma cantidad
- Ninguno tiene missing
- Los tipos tambien es interesante observar

In [9]:
(data['rating'].isna()).sum()

0

In [10]:
# Find the missing value
item['release_date'].unique()
item.index[item[item['release_date']==None].any(axis=1)].tolist()

#item.loc[item['release_date'].isna() | (len(item['release_date'])<2), 'release_date']

[]

In [11]:
user['gender'].value_counts()

M    670
F    273
Name: gender, dtype: int64

In [12]:
# Detect users with lot of votes
lotOfVotes = 70

rating_amount = data.groupby("user_id").count()
print(rating_amount[rating_amount['rating'] > lotOfVotes].head())

         movie_id  rating  timestamp
user_id                             
1             135     135        135
5              91      91         91
6             110     110        110
7             213     213        213
10             94      94         94


In [13]:
# Detect users with few votes
data['movie_id'].value_counts().tail(n=10)

1307    1
1371    1
1627    1
1436    1
1500    1
1675    1
1564    1
1460    1
1332    1
1663    1
Name: movie_id, dtype: int64

In [14]:
#train.groupby("userId")["rating"].transform("median")
userId = 36
user_training = data[data['user_id'] == userId]
print("User avg rating", user_training.groupby("user_id")["rating"].transform("mean").iloc[0])
#user_training['user-ranking'].iloc[0]

print(data[data['user_id'] == userId])

User avg rating 4
      user_id  movie_id  rating  timestamp
2232       36       261       5  882157581
2233       36       288       4  882157227
2234       36       289       2  882157356
2235       36       333       4  882157227
2236       36       358       5  882157581
2237       36       873       3  882157386
2238       36       875       3  882157470
2239       36       878       5  882157581
2240       36      1026       5  882157581


In [15]:
data.movie_id.tolist()

[1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 11,
 13,
 15,
 16,
 18,
 19,
 21,
 22,
 25,
 26,
 28,
 29,
 30,
 32,
 34,
 35,
 37,
 38,
 40,
 41,
 42,
 43,
 45,
 46,
 48,
 50,
 52,
 55,
 57,
 58,
 59,
 63,
 66,
 68,
 71,
 75,
 77,
 79,
 83,
 87,
 88,
 89,
 93,
 94,
 95,
 99,
 101,
 105,
 106,
 109,
 110,
 111,
 115,
 116,
 119,
 122,
 123,
 124,
 126,
 127,
 131,
 133,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 146,
 147,
 149,
 152,
 153,
 156,
 158,
 162,
 165,
 166,
 167,
 168,
 169,
 172,
 173,
 176,
 178,
 179,
 181,
 182,
 187,
 191,
 192,
 194,
 195,
 197,
 198,
 199,
 203,
 204,
 205,
 207,
 211,
 216,
 217,
 220,
 223,
 231,
 234,
 237,
 238,
 239,
 240,
 244,
 245,
 246,
 247,
 249,
 251,
 256,
 257,
 261,
 263,
 268,
 269,
 270,
 271,
 1,
 10,
 14,
 25,
 100,
 111,
 127,
 237,
 242,
 255,
 258,
 269,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 291,
 293,
 294,
 295,
 296,
 300,
 302,
 304,
 305,
 306,
 309,
 310,
 311,
 181,
 258,
 260,

In [16]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 31/12/2022 09:42:35
